In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# Read raw data
food_insecurity_updated = pd.read_csv('https://raw.githubusercontent.com/datanerddhanya/DATA608/refs/heads/main/df_clean_summary_code.csv')

# Define income order for proper sorting
income_order = [
    'Less than $5,000',
    '$5,000 to $7,499',
    '$7,500 to $9,999',
    '$10,000 to $12,499',
    '$12,500 to $14,999',
    '$15,000 to $19,999',
    '$20,000 to $24,999',
    '$25,000 to $29,999',
    '$30,000 to $34,999',
    '$35,000 to $39,999',
    '$40,000 to $49,999',
    '$50,000 to $59,999',
    '$60,000 to $74,999',
    '$75,000 to $99,999',
    '$100,000 to $149,999',
    '$150,000 or more'
]

# Ensure the income column is a categorical type with the specified order
food_insecurity_updated['family_income'] = pd.Categorical(
    food_insecurity_updated['family_income'], 
    categories=income_order, 
    ordered=True
)

# Calculate food insecurity by state using weighted sums
state_stats = (food_insecurity_updated.groupby('state')
                    .apply(lambda x: pd.Series({
                        'food_insecurity_ratio': (x.loc[x['food_security_status'] != 'Food Secure', 'person_supplement_weight'].sum() / 
                                                  x.loc[x['food_security_status'] == 'Food Secure', 'person_supplement_weight'].sum() * 100)
                    }))
              )

# Round the 'food_insecurity_ratio' column in the summary dataframe
state_stats['food_insecurity_ratio'] = round(state_stats['food_insecurity_ratio'], 2)

display(state_stats)


# Convert food security status to binary (insecure = 1, secure = 0)
food_insecurity_updated['is_food_insecure'] = (food_insecurity_updated['food_security_status'] != 'Food Secure').astype(int)
    

# Analysis 1: Food insecurity by state and age group
state_age_stats = (food_insecurity_updated.groupby(['state', 'age_group'])
                            .apply(lambda x: pd.Series({
                                'pct_food_insecure': (x['is_food_insecure'] * 
                                                     x['person_supplement_weight']).sum() / 
                                                     x['person_supplement_weight'].sum() * 100,
                                'sample_size': len(x)
                            }))
                            .round(2))

state_age_stats = state_age_stats.reset_index()

# Flatten column names if they are hierarchical
state_age_stats.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in state_age_stats.columns]

display(state_age_stats)  
# Analysis 2: Food insecurity by state and demography
state_demo_stats = (food_insecurity_updated.groupby(['state', 'demography'])
                            .apply(lambda x: pd.Series({
                                'pct_food_insecure': (x['is_food_insecure'] * 
                                                     x['person_supplement_weight']).sum() / 
                                                     x['person_supplement_weight'].sum() * 100,
                                'sample_size': len(x)
                            }))
                            .round(2))


state_demo_stats = state_demo_stats.reset_index()

# Flatten column names if they are hierarchical
state_demo_stats.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in state_demo_stats.columns]


display(state_demo_stats)

# Analysis 3: Food insecurity by income level
food_insecurity_updated['income_level'] = pd.Categorical(food_insecurity_updated['family_income'], categories=income_order, ordered=True)
income_insecurity = (food_insecurity_updated.groupby('income_level')
                          .apply(lambda x: pd.Series({
                              'pct_food_insecure': (x['is_food_insecure'] * 
                                                   x['person_supplement_weight']).sum() / 
                                                   x['person_supplement_weight'].sum() * 100
                              
                          }))
                          .round(2))


display(income_insecurity)

,food_insecurity_ratio
state,
Alabama,12.37
Alaska,16.13
Arizona,14.18
Arkansas,28.06
California,16.52
Colorado,14.88
Connecticut,12.59
Delaware,13.48
District of Columbia,7.49


,state,age_group,pct_food_insecure,sample_size
0,Alabama,(0-17),14.49,34.0
1,Alabama,(18-34),12.80,59.0
2,Alabama,(35-44),10.30,45.0
3,Alabama,(45-54),15.28,55.0
4,Alabama,(55-64),6.88,38.0
...,...,...,...,...
301,Wyoming,(18-34),13.71,54.0
302,Wyoming,(35-44),19.64,45.0
303,Wyoming,(45-54),14.19,39.0
304,Wyoming,(55-64),12.74,37.0


,state,demography,pct_food_insecure,sample_size
0,Alabama,Children,14.49,34.0
1,Alabama,Men,8.20,111.0
2,Alabama,Women,11.73,134.0
3,Alaska,Children,18.79,30.0
4,Alaska,Men,11.19,95.0
...,...,...,...,...
148,Wisconsin,Men,8.57,94.0
149,Wisconsin,Women,10.72,108.0
150,Wyoming,Children,23.89,34.0
151,Wyoming,Men,13.18,108.0


,pct_food_insecure
income_level,
"Less than $5,000",34.37
"$5,000 to $7,499",32.52
"$7,500 to $9,999",34.62
"$10,000 to $12,499",39.13
"$12,500 to $14,999",33.01
"$15,000 to $19,999",34.97
"$20,000 to $24,999",33.52
"$25,000 to $29,999",27.05
"$30,000 to $34,999",26.00


In [6]:
state_demo_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   state              153 non-null    object 
 1   demography         153 non-null    object 
 2   pct_food_insecure  153 non-null    float64
 3   sample_size        153 non-null    float64
dtypes: float64(2), object(2)
memory usage: 4.9+ KB


In [2]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Function to update the plot based on selected state
def update_plot(state):
    filtered_demo_df = state_demo_stats[state_demo_stats['state'] == state]
    filtered_age_df = state_age_stats[state_age_stats['state'] == state]
    
    fig_demo = px.bar(filtered_demo_df, x='demography', y='pct_food_insecure', 
                      title=f'Food Insecurity Rate by Demography in {state}',
                      labels={'pct_food_insecure': 'Food Insecurity Rate (%)', 'demography': 'Demography'})
    
    fig_age = px.bar(filtered_age_df, x='age_group', y='pct_food_insecure', 
                     title=f'Food Insecurity Rate by Age Group in {state}',
                     labels={'pct_food_insecure': 'Food Insecurity Rate (%)', 'age_group': 'Age Group'})
    
    fig_demo.show()
    fig_age.show()

# Create a dropdown widget for state selection
state_dropdown = widgets.Dropdown(
    options=state_demo_stats['state'].unique(),
    description='State:',
    value=state_demo_stats['state'].unique()[0]
)

# Function to display the dropdown and update the plot
def display_widgets():
    display(state_dropdown)
    widgets.interactive(update_plot, state=state_dropdown)

# Call the function to display the dropdown and the initial plot
display_widgets()

Dropdown(description='State:', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', …